## IMPORTING LIBRARIES

In [1]:
%pip install tensorflow opencv-python matplotlib

import os
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

Note: you may need to restart the kernel to use updated packages.


## LOADING TRAINING DATA AND SPLIT DATASET

In [6]:
train_dir = "asl_alphabet_train"
test_dir = "asl_alphabet_test"

# Data Augmentation for Training
datagen = ImageDataGenerator(
    rescale=1./255,    
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  
)

# Training Data
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    subset='training'
)

# Validation Data
validation_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation'
)

class_labels = list(train_generator.class_indices.keys())
print(f"Classes: {class_labels}")


Found 62400 images belonging to 26 classes.
Found 15600 images belonging to 26 classes.
Classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


## BUILDING CNN MODEL

In [7]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(26, activation='softmax') 
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 26)             │         3,354 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 685,978 (2.62 MB)

 Trainable params: 685,978 (2.62 MB)

 Non-trainable params: 0 (0.00 B)

## TRAIN THE MODEL

In [9]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    verbose=1
)

model.save("asl_sign_language_model.h5")
print("Model saved successfully!")

Epoch 1/20
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 845s 433ms/step - accuracy: 0.5240 - loss: 1.4253 - val_accuracy: 0.5051 - val_loss: 1.6546
Epoch 2/20
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 104s 53ms/step - accuracy: 0.6639 - loss: 0.9840 - val_accuracy: 0.6003 - val_loss: 1.3230
Epoch 3/20
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 103s 53ms/step - accuracy: 0.7358 - loss: 0.7579 - val_accuracy: 0.6066 - val_loss: 1.2966
Epoch 4/20
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 100s 51ms/step - accuracy: 0.7859 - loss: 0.6168 - val_accuracy: 0.6476 - val_loss: 1.1932
Epoch 5/20
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 103s 53ms/step - accuracy: 0.8173 - loss: 0.5335 - val_accuracy: 0.6671 - val_loss: 1.2286
Epoch 6/20
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 136s 70ms/step - accuracy: 0.8396 - loss: 0.4640 - val_accuracy: 0.6811 - val_loss: 1.1939
Epoch 7/20
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 251s 129ms/step - accuracy: 0.8560 - loss: 0.4259 - val_accuracy: 0.7008 - val_loss: 1.0708
Epoch 8/20
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 296s 146ms/step - accurac

Model saved successfully!


## LOADING MODEL AND TESTING ON ASL DATASET

In [10]:
model = load_model("asl_sign_language_model.h5")

def predict_gesture(img_path, model):
    img = image.load_img(img_path, target_size=(64, 64), color_mode="grayscale")
    img_array = image.img_to_array(img) / 255.0  
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    
    print(f"Predicted Gesture: {class_labels[predicted_class]}")
    return class_labels[predicted_class]

test_images = os.listdir(test_dir)
for img_name in test_images:
    img_path = os.path.join(test_dir, img_name)
    predicted_gesture = predict_gesture(img_path, model)
    print(f"File: {img_name} | Prediction: {predicted_gesture}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Predicted Gesture: A
File: A_test.jpg | Prediction: A
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted Gesture: B
File: B_test.jpg | Prediction: B
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted Gesture: C
File: C_test.jpg | Prediction: C
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted Gesture: D
File: D_test.jpg | Prediction: D
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted Gesture: E
File: E_test.jpg | Prediction: E
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted Gesture: F
File: F_test.jpg | Prediction: F
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted Gesture: G
File: G_test.jpg | Prediction: G
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted Gesture: H
File: H_test.jpg | Prediction: H
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted Gesture: I
File: I_test.jpg | Prediction: I
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicted Gesture: J
File: J_test.jpg | Prediction: J
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Predicted Gesture: K
File: K_test.jpg | Pr

## ACCURACY : BASED ON VALIDATION DATA

In [11]:
# Evaluate the model on validation data
loss, accuracy = model.evaluate(validation_generator, verbose=1)
accuracy_percent = accuracy * 100
print(f"Model Accuracy: {accuracy_percent:.2f}%")

488/488 ━━━━━━━━━━━━━━━━━━━━ 15s 30ms/step - accuracy: 0.7250 - loss: 1.4152
Model Accuracy: 72.54%


## REAL TIME DETECTION USING WEBCAM

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (64, 64)) / 255.0
    img_array = np.expand_dims(resized, axis=(0, -1))

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    
    cv2.putText(frame, class_labels[predicted_class], (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Real-Time Sign Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41